# Get Started with Data for Good Datasets

This guide assumes you have already followed the steps [here](https://github.com/databricks-solutions/dais-2025-genai-hackathon?tab=readme-ov-file#1-set-up-your-workspace-and-get-data) to set up your workspace and send your Delta Sharing ID to request the partner data. It might take some time for the data to show up in the Delta Sharing section of your workspace. Once it is there, [add it as a catalog so you can use it in your workspace](https://github.com/databricks-solutions/dais-2025-genai-hackathon?tab=readme-ov-file#accessing-the-data).

You will have access to three schemas:
- `bright_initiative`
- `mimilabs`
- `nimble`

with the data provided by each of these organizations.

## How to Use This Guide
The goal of this guide is to help you with the first steps of accessing these data and using them with AI tools. Your actual projects will most likely not look like these examples, but we hope showing how to start working with the data will help you get started!

## Recommended Approach
1. Follow the instruction [here](https://github.com/databricks-solutions/dais-2025-genai-hackathon?tab=readme-ov-file#1-set-up-your-workspace-and-get-data) to set up your workspace and get the data.
2. Pick a theme or challenge you are interested in.
3. Spend some time [exploring the data](#exploring-the-data) with AI/BI Genie spaces.
4. Determine what your application needs to do with the data in order to solve the challenge you have chosen to pursue.
5. Pick the framework or tools that are the best fit for your application.
6. Build!

## Exploring the Data

To start exploring the data, we recommend creating one or more [AI/BI Genie Spaces](https://docs.databricks.com/aws/en/genie). Genie spaces let you ask natural language questions about the data you're interested in and get SQL queries, answers, and visualizations in return. It is a great way to quickly get up to speed on new data. You can access AI/BI Genie in the left sidebar in the `SQL` section.


## Quickstart Code Snippets

Here are some short code snippets showing how to start using the data once you've added the catalog.

### Setup

In [0]:
%pip install -U -qqq langchain_core langchain_databricks langchain_community
%restart_python

### Bright Initiative

The Bright Initiative has provided the following data:
- **U.S. Google Maps Businesses Dataset - 5M Records:** Access detailed information on businesses across diverse industries in the U.S., including names, addresses, contact details, and ratings. 
- **Global Booking.com Hotel Listings Dataset: 1.6M Records:** The Booking dataset provides detailed and structured information on global accommodations, including hotel descriptions, location details, pricing, amenities, and guest reviews. It offers critical insights into property features, pricing trends, and customer preferences.
- **Global Airbnb Properties Information Dataset: 1.6M Records:** Uncover the details of global Airbnb properties with this comprehensive dataset, providing vital information for travelers, property managers, and researchers. Access key property features such as price, images, descriptions, availability, reviews, and host details.

Let's look at the Airbnb Properties Dataset. First, we'll start with a simple SQL query. You can query the datasets directly from a notebook or in the SQL Editor. Note that there are two different versions of the Bright Initiave datasets, differing mostly in whether nested columns are formatted as strings or arrays. Feel free to use whichever is more convenient for your use case.

Here's how we might check the reviews of different Airbnb locations for mentions of wheelchair accessibility:

In [0]:
%sql
SELECT * FROM `dais-hackathon-2025`.bright_initiative.airbnb_properties_information
WHERE
  location = 'Chicago, Illinois, United States'
  AND host_number_of_reviews > 1000
  AND EXISTS(reviews, review -> review ILIKE '%wheelchair%')
LIMIT 5;

#### Integrate with Agents

Here is a naive approach to using these data with agents. We look for listings with reviews mentioning wheelchairs, extract a few fields with information about the listings (including reviews), and ask the model to describe the accessibility features of the listings.

To expand on this approach and make it more suitable for your project, consider:
- Letting the agent write the SQL for you, which allows it to answer flexible questions like "are there any listings with a pool?" without you needing to write a new query for every possible feature.
- Combining keyword search with structured data for more powerful retrieval, so you can find reviews that mention "spacious" but only for listings with a guest rating above 4.8.
- Giving your agent multiple "tools" to choose from, such as one function to query the Airbnb data and another to fetch live hotel prices from a different partner's API.
- Integrate with other Bright data sources for a more holistic view of neighborhood accessibility, or for comparisons with other lodging options such as hotels.

There are many tools, both from LangChain and Databricks, that can help with these steps!

In [0]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

# configure workspace tokens
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

def format_context(df: pd.DataFrame) -> str:
    """
    Converts the DataFrame into a JSON string to ensure all data is passed
    to the model without truncation. JSON is also a great format for structured data
    like you have in 'description_by_sections'.
    """
    return df.to_json(orient='records', indent=2)

def find_accessible_airbnb_properties(location: str) -> pd.DataFrame:
  """
  Queries the Bright Initiative Airbnb dataset for properties in a specific location
  that have reviews mentioning "wheelchair".
  """
  query = f"""
    SELECT
      listing_name,
      location_details,
      location,
      details,
      description_by_sections,
      reviews
    FROM `dais-hackathon-2025`.bright_initiative.airbnb_properties_information
    WHERE
      location ILIKE '%{location}%'
      AND host_number_of_reviews > 1000
      AND EXISTS(reviews, review -> review ILIKE '%wheelchair%')
    LIMIT 5
  """
  return format_context(spark.sql(query).toPandas())

# Define the prompt template for the LLM
prompt_template = PromptTemplate.from_template(
  """
  You are a helpful assistant for accessible travel. Your goal is to summarize potential Airbnb listings for a user.

  The following listing *mention* wheelchairs but may not actually be accessible. Closely review the descriptions and review,
  and then summarize the accessibility features (or lack thereof).

  Here is the JSON data:
  {context}
  """
)

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

# This is our simple "agentic" chain
chain = (
    find_accessible_airbnb_properties
    | prompt_template
    | llm
    | StrOutputParser()
)

# Let's run the chain for Chicago!
result = chain.invoke("Chicago")

print(result)

### mimilabs

First, watch [this video](https://www.youtube.com/watch?v=2MYZ5WnrqDk) for help getting started. You can also use [mimibot](https://www.mimilabs.ai/mimibot) to ask questions about the provided data. The Delta Share with the data also includes two sample notebooks showing how to use these data—well worth consulting!

These data are suitable for, among other things, building a "benefits navigation agent". Choosing a Medicare Advantage plan can be difficult and overwhelming, and there are a lot of opportunities for AI agents to help.

First, let's set up the data as demonstrated in the example notebook provided via the Delta share.

In [0]:
# This query processes the raw PBP data and creates a clean, analysis-ready temporary view.
spark.sql("""
CREATE OR REPLACE TEMP VIEW medicare_benefits_analysis AS
WITH benefit_codes AS (
  SELECT array(
    '14c10', '14c11', '14c12', '14c13', '14c14', '14c15', '14c16', '14c17', '14c18', '14c19',
    '14c20', '14c21', '14c22', '14c8', '14c9', '17a1', '17a2', '17b1', '17b2', '17b3',
    '17b4', '17b5', '18a1', '18a2', '18b1', '18b2', '18b3', '18b4', '18c',
    '13a', '7b1', '7b2', '16b1', '16b2', '16b3', '16b4', '16b5', '16b6',
    '11a1', '11a2', '14c4', '6-1', '6-2', '6-3', '6-4', '13b',
    '13i1', '13i2', '13i3', '13i4', '13i5', '13i6', '13i7', '13i8', '13i9', '13i10',
    '13i11', '13i12', '13i13', '13i14', '13i15', '10b1', '10b2'
  ) AS codes
),
latest_bids AS (
  SELECT *
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY bid_id ORDER BY version DESC) as rn
    FROM `dais-hackathon-2025`.mimilabs.pbp_section_d
    WHERE YEAR(mimi_src_file_date) = 2025
  )
  WHERE rn = 1
),
processed_bids AS (
  SELECT
    lb.*,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_1, ''), '\\s+', ''), ';'), x -> x != '') AS clean_cats_1,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_2, ''), '\\s+', ''), ';'), x -> x != '') AS clean_cats_2,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_3, ''), '\\s+', ''), ';'), x -> x != '') AS clean_cats_3,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_4, ''), '\\s+', ''), ';'), x -> x != '') AS clean_cats_4,
    filter(split(regexp_replace(COALESCE(lb.pbp_d_combo_nmc_cats_5, ''), '\\s+', ''), ';'), x -> x != '') AS clean_cats_5
  FROM latest_bids lb
  CROSS JOIN benefit_codes bc
  WHERE arrays_overlap(bc.codes,
    filter(split(regexp_replace(CONCAT_WS(';',
          COALESCE(lb.pbp_d_combo_nmc_cats_1, ''), COALESCE(lb.pbp_d_combo_nmc_cats_2, ''),
          COALESCE(lb.pbp_d_combo_nmc_cats_3, ''), COALESCE(lb.pbp_d_combo_nmc_cats_4, ''),
          COALESCE(lb.pbp_d_combo_nmc_cats_5, '')), '\\s+', ''), ';'), x -> x != '')
  )
),
final_benefits AS (
  SELECT
    pb.pbp_a_hnumber,
    pb.pbp_a_plan_identifier,
    array_distinct(flatten(array(pb.clean_cats_1, pb.clean_cats_2, pb.clean_cats_3, pb.clean_cats_4, pb.clean_cats_5))) AS all_benefits
  FROM processed_bids pb
)
SELECT
  pbp_a_hnumber,
  pbp_a_plan_identifier,
  all_benefits,
  arrays_overlap(all_benefits, array('13a')) AS has_acupuncture,
  arrays_overlap(all_benefits, array('7b1', '7b2')) AS has_chiropractic,
  arrays_overlap(all_benefits, array('16b1', '16b2', '16b3', '16b4', '16b5', '16b6')) AS has_dental,
  arrays_overlap(all_benefits, array('11a1', '11a2')) AS has_dme,
  arrays_overlap(all_benefits, array('14c4')) AS has_fitness,
  arrays_overlap(all_benefits, array('6-1', '6-2', '6-3', '6-4')) AS has_home_health,
  arrays_overlap(all_benefits, array('13b', '13i1', '13i2', '13i3', '13i4', '13i5', '13i6', '13i7', '13i8', '13i9', '13i10', '13i11', '13i12', '13i13', '13i14', '13i15')) AS has_otc_ssbci,
  arrays_overlap(all_benefits, array('10b1', '10b2')) AS has_transportation
FROM final_benefits
""")

print("Temporary view 'medicare_benefits_analysis' created successfully.")


Let's find plans that offer fitness benefits.

In [0]:
%sql
-- Find the first 10 plans that offer fitness benefits
SELECT *
FROM
  medicare_benefits_analysis
WHERE
  has_fitness = true
LIMIT 10;

#### Integrate with Agents
Here is a simple approach to using these data in the context of the agents hackathon. To expand on this approach:
- Using [mimibot](https://www.mimilabs.ai/mimibot), explore how these data relate to other tables provided by mimilabs, enabling you to further enrich the data
- Explore other tools you could use that would enable more flexible retrieval than a hardcoded sql query
- Connect to a web search tool that can enrich these data with user-friendly expalantions of benefits and what they mean

In [0]:
import pandas as pd
from typing import List
from langchain_community.tools import tool
from langchain_databricks import ChatDatabricks
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from databricks.sdk import WorkspaceClient
import os
import ast

@tool
def find_plans_with_benefits(required_benefits: str) -> str:
  """
  Finds Medicare Advantage plans that include all of the specified supplemental benefits.
  Use this tool when a user asks to find plans with a specific list of benefits.
  The input must be a Python list of benefit names provided as a string, chosen from:
  'acupuncture', 'chiropractic', 'dental', 'dme', 'fitness', 'home_health', 'otc_ssbci', 'transportation'.
  For example: "['dental', 'fitness']"
  """
  try:
    # The LLM often returns a string representation of a list, e.g., "['dental', 'fitness']".
    # ast.literal_eval safely evaluates this string into an actual Python list.
    benefits_list = ast.literal_eval(required_benefits)
    if not isinstance(benefits_list, list):
        raise ValueError("Input could not be parsed into a list.")
  except (ValueError, SyntaxError) as e:
    # If parsing fails, return an error message to the agent so it can retry.
    return f"Error: Invalid input format. Expected a string representation of a list. {e}"

  # Build a WHERE clause from the list of required benefits
  where_conditions = [f"has_{benefit.strip()} = true" for benefit in benefits_list]
  where_clause = " AND ".join(where_conditions)

  query = f"""
    SELECT pbp_a_hnumber, pbp_a_plan_identifier, all_benefits
    FROM medicare_benefits_analysis
    WHERE {where_clause}
    LIMIT 10
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)

# Configure workspace client for authentication
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

# The LLM is the "brain" of the agent
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=2048)

# The Agent needs a list of tools it can use
tools = [find_plans_with_benefits]

# This prompt template tells the agent how to reason about using tools
# This is a standard "ReAct" (Reasoning + Acting) prompt
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Create the agent by combining the LLM, tools, and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask our agent a question in natural language!
response = agent_executor.invoke({
    "input": "Can you find me some plans that have dental, fitness, and transportation benefits?"
})

print("\n\n--- Final Answer ---")
print(response['output'])


### Nimble

See [this page](https://nimbleway.notion.site/Hackathon-Resource-Center-209c8a32950080f785e6cecc1e502a8b) for details on getting started with Nimble's MCP server and advice for using Nimble in the hackathon, and [this notebook](https://github.com/databricks-solutions/dais-2025-genai-hackathon/blob/main/2025_agent_hackathon_resources/nimble-mcp.ipynb) for instructions on using Nimble's MCP server with Databricks model serving via LangGraph and/or LlamaIndex.

The Nimble team has also provided access to structured data from a variety of sources, such as amazon, footlocker, walmart, and yelp.

Here's an example of working with the Amazon product data.

In [0]:
%sql
select * from `dais-hackathon-2025`.nimble.dbx_amazon_serp_daily limit 5;

#### Integrate with Agents

Here is a simple way to set up an agent to come up with multiple search terms and query the amazon products table for products to solve a given problem. To extend this for your hackathon project, you could:
- Follow the links to get more information on the specific Amazon products
- Obtain additional data sources using the Nimble MCP server
- Use a multimodal model to evaluate the product images

In [0]:
import pandas as pd
from langchain_community.tools import tool
from langchain_databricks import ChatDatabricks
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from databricks.sdk import WorkspaceClient
import os


@tool
def find_products_by_keyword(search_term: str) -> str:
  """
  Searches the Nimble e-commerce dataset for products matching a keyword search_term.
  Use this to find products based on a user's request.
  The input must be a single string. For example: "clip on reading light"
  """
  query = f"""
    SELECT
      productId,
      product_name,
      price,
      rating,
      review_count,
      product_url
    FROM `dais-hackathon-2025`.nimble.dbx_amazon_serp_daily
    WHERE
      product_name ILIKE '%{search_term}%'
    LIMIT 5
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)

  w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

# The LLM is the "brain" of the agent
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=2048)

# The Agent needs a list of tools it can use
tools = [find_products_by_keyword]

# This prompt template tells the agent how to reason and use tools
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Your process should be to first think of a few different, specific search terms based on the user's question.
Then, use the `find_products_by_keyword` tool for each of those keywords, one at a time.
After you have gathered all the product information from your searches, combine the results into a single, helpful summary for the user.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: After you find the products from all your searches, your final answer should summarize them and suggest that a good next step would be to use a web browsing tool to analyze the product pages for more detailed information.

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Create the agent by combining the LLM, tools, and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask our agent to find a product related to accessibility
response = agent_executor.invoke({
    "input": "I'm looking for a tool to help my elderly parent open tight jars."
})

print("\n\n--- Final Answer ---")
print(response['output'])